<a href="https://colab.research.google.com/github/SIVASKANDAN-P/PYTHON-CODE-HINT/blob/main/model_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install datasets
!pip install -U datasets
!pip install bert_score
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install trl
!pip install datasets


In [29]:
#importing necessary libraries
from datasets import load_dataset, load_dataset_builder
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from bert_score import score
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
#    prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    PeftModel
)
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from transformers import T5Tokenizer, T5ForConditionalGeneration
from trl import SFTTrainer
from datasets import Dataset

# Connecting to gdrive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!pip install evaluate
!pip install rouge_score
#In the cell "ipython-input-4-99b29ab04d12", add this line at the beginning of the cell:
import evaluate

In [31]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [32]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response:

    ### Task:
    Break down the give coding problem into its logical steps

    ### Input:
    {sample['instruction']}

    ### Response:
    {sample['response']}
    """

def tokenize_function(sample):
    return tokenizer(sample['instruction'], padding="max_length", truncation=True)

# def compute_metrics(eval_pred):
#     """Computes and returns a dictionary of metrics (e.g., accuracy, f1)."""


#     # Unpack predictions and labels
#     predictions, labels = eval_pred
#     print(predictions)
#     print(labels)
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # Handle potential differences in labels format:
#     if labels.ndim == 2 and labels.shape[1] == tokenizer.model_max_length: # If token IDs
#         # Decode if token IDs & reshape if needed
#         decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     else:
#         # Assuming decoded labels, maybe list of lists or strings
#         if isinstance(labels[0], list): # list of lists of word tokens
#             decoded_labels = [" ".join(label_tokens) for label_tokens in labels]
#         else:  #list of strings
#             decoded_labels = labels
#     # Ensure decoded_labels is a list of strings (one per example) and matches decoded_preds
#     if not isinstance(decoded_labels[0], str):
#         decoded_labels = [str(label) for label in decoded_labels]

#     decoded_labels = [decoded_labels] * len(decoded_preds)  # Duplicate for each pred if needed

#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     # Define and calculate desired metrics using 'evaluate' library
#     metric = evaluate.load("rouge")
#     # result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)

#     return result

In [33]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Replace -100 in labels with tokenizer's padding token ID
    labels = np.where(labels != -100, labels, loaded_tokenizer.pad_token_id)

    # Replace -100 in predictions with tokenizer's padding token ID
    predictions = np.where(predictions != -100, predictions, loaded_tokenizer.pad_token_id)
    decoded_predictions = loaded_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = loaded_tokenizer.batch_decode(labels, skip_special_tokens=True)

    metric = evaluate.load("rouge")
    return metric.compute(predictions=decoded_predictions, references=decoded_labels)


In [6]:
model1 = "google/flan-t5-large"
model2 = "google/flan-t5-small"
model3 = "google/flan-t5-base"
model4 = "google/flan-t5-xl"
model5 = "google/flan-t5-xxl"
model6 = "codellama/CodeLlama-7b-hf"

In [7]:
base_model = model6

In [34]:
try:
  tokenizer = AutoTokenizer.from_pretrained(base_model)
except:
  tokenizer = T5Tokenizer.from_pretrained(base_model)

In [9]:
dataset = pd.read_csv("/content/drive/MyDrive/dataset/data300.csv")
dataset = dataset[['instruction','response']]
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)
# train_dataset_tokenised Pandas DataFrame to hugging face dataset
train_dataset_hf = Dataset.from_pandas(train_dataset)
train_dataset_tokenized = train_dataset_hf.map(tokenize_function, batched=True)
# train_dataset_tokenised Pandas DataFrame to hugging face dataset
test_dataset_hf = Dataset.from_pandas(test_dataset)
test_dataset_tokenized = test_dataset_hf.map(tokenize_function, batched=True)

Map:   0%|          | 0/18706 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/4677 [00:00<?, ? examples/s]

In [35]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4")

In [37]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules="all-linear",  # To apply LoRA to all the linear layers
    lora_dropout=0.05,
    bias="none",                  #since for other when adopter is removed model will not perform the same
    task_type="CAUSAL_LM"         #since it is text generation task
)

#model = T5ForConditionalGeneration.from_pretrained(
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    #use_gradient_checkpointing=True
    quantization_config=quantization_config
)
peft_model = get_peft_model(model, config)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# model = T5ForConditionalGeneration.from_pretrained(
#     base_model,
#     #use_gradient_checkpointing=True
#     quantization_config=quantization_config
# )

using evaluation_strategy="no" for a training run without intermediate evaluation and then running the evaluation separately afterward is a valid and effective approach, especially if you're dealing with memory constraints. Here’s why this can be a good strategy:

Benefits of Running Evaluation Separately:
Reduced Memory Usage During Training: Skipping intermediate evaluations during training frees up GPU memory, allowing you to allocate more resources to the training process.
Simpler Resource Management: You can focus on optimizing memory and computational resources for training without worrying about the extra load that evaluations may introduce.
Controlled Evaluation Process: Running the evaluation separately allows you to better manage the evaluation environment, potentially using different settings or even moving the model to the CPU if GPU memory is a concern.

In [38]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/model',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    logging_dir="/content/drive/MyDrive/model/log",
#    evaluation_strategy="steps",
    evaluation_strategy="no",    # Due to memory constraint restricting evaluation so we can run later.
    save_strategy="epoch",
    eval_steps=1000,
    save_steps=1000,
    per_device_eval_batch_size=1,  # Reduce eval batch size to reduce memory during evaluation
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:



# Create the Trainer instance with SFTTrainer
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset_tokenized, # Your tokenized dataset
    compute_metrics = compute_metrics,
    eval_dataset=test_dataset_tokenized            # Optional validation dataset
    # data_collator=data_collator, # Optional data collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
 

In [40]:
torch.cuda.empty_cache()

In [41]:
trainer.train()

Step,Training Loss
500,1.286500
1000,1.170000
1500,1.095600
2000,1.030500
2500,1.038800
3000,0.992100
3500,0.894700
4000,0.907600
4500,0.882000
5000,0.872100


TrainOutput(global_step=9353, training_loss=0.9084544374026883, metrics={'train_runtime': 12912.8417, 'train_samples_per_second': 1.449, 'train_steps_per_second': 0.724, 'total_flos': 9.49291008385106e+16, 'train_loss': 0.9084544374026883, 'epoch': 1.0})

In [42]:
import datetime
# Get the current timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
print(timestamp)
# Save the model to a specified directory
trainer.save_model(os.path.join('/content/drive/MyDrive/model', f"{base_model}_{timestamp}"))

# Optionally, save the tokenizer as well
trainer.tokenizer.save_pretrained(os.path.join('/content/drive/MyDrive/model', f"{base_model}_token_{timestamp}"))

20241107_150314


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_token_20241107_150314/tokenizer_config.json',
 '/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_token_20241107_150314/special_tokens_map.json',
 '/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_token_20241107_150314/tokenizer.model',
 '/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_token_20241107_150314/added_tokens.json',
 '/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_token_20241107_150314/tokenizer.json')

In [43]:
model_path = os.path.join('/content/drive/MyDrive/model', f"{base_model}_{timestamp}")
tokeniser_path = os.path.join('/content/drive/MyDrive/model', f"{base_model}_token_{timestamp}")

In [51]:
def log_data(base_model,TrainingArguments,LoraConfig,quantization_config,model_path,tokeniser_path):
  data = pd.read_csv("/content/drive/MyDrive/dataset/results.csv")
  results = dict()
  results['Model'] = base_model
  results['tim-stmp'] = timestamp
  results['Epoch'] = TrainingArguments.num_train_epochs
  results['Learning_rate'] = TrainingArguments.learning_rate
  results['Lora_rank'] = LoraConfig.r
  results['Lora_alpha'] = LoraConfig.lora_alpha
  results['quant_4bit'] = quantization_config.load_in_4bit
  results['quant_8bit'] = quantization_config.load_in_8bit
  results['Trainable_parameters'] = peft_model.print_trainable_parameters()
  #results['Trainable_parameters'] = "trainable params: 39,976,960 || all params: 6,778,523,648 || trainable%: 0.5898 data saved"
  results['model_path'] = model_path
  results['tokeniser_path'] = tokeniser_path
  results['train_loss']= 0.9084544374026883
  df = pd.DataFrame.from_dict([results])
  fin = pd.concat([data, df], ignore_index=True)
  fin.to_csv("/content/drive/MyDrive/dataset/results.csv")
  print("data saved" )

In [50]:
log_data(base_model,training_args,config,quantization_config,model_path,tokeniser_path)

data saved


Loading the model and tokeniser for evaluation


In [21]:
torch.cuda.empty_cache()

In [14]:
torch.cuda.empty_cache()
# Load the model and tokenizer
loaded_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_token_20241107_032806')
loaded_model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_20241107_032806',quantization_config=quantization_config)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [18]:
trainer = SFTTrainer(
    model=loaded_model,
    args=training_args,
    train_dataset=train_dataset_tokenized, # Your tokenized dataset
    compute_metrics = compute_metrics,
    eval_dataset=test_dataset_tokenized            # Optional validation dataset
    # data_collator=data_collator, # Optional data collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
 

In [21]:
chunk = trainer.eval_dataset[0:2]
chunk_dataset = Dataset.from_dict(chunk)
trainer.predict(chunk_dataset)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


PredictionOutput(predictions=array([[[-6.3554688e+00, -7.6210938e+00,  3.1972656e+00, ...,
         -7.4023438e-01,  1.6611328e+00, -7.3828125e-01],
        [-2.6269531e+00, -2.7519531e+00, -1.3798828e+00, ...,
         -2.0000000e+00,  5.4248047e-01,  4.1046143e-02],
        [-5.3320312e+00, -5.0781250e+00,  5.7910156e-01, ...,
          4.3261719e-01, -5.2636719e-01,  1.1474609e+00],
        ...,
        [-3.6269531e+00, -2.7740479e-02,  9.4843750e+00, ...,
         -5.2404404e-04,  1.3076172e+00,  5.1708984e-01],
        [-5.5703125e+00, -1.2177734e+00,  5.4882812e+00, ...,
          8.3740234e-01,  3.8696289e-02, -2.5097656e-01],
        [-4.1523438e+00,  1.6767578e+00,  8.1562500e+00, ...,
         -5.1765442e-03,  6.0937500e-01, -7.9492188e-01]],

       [[-6.3515625e+00, -7.6132812e+00,  3.1914062e+00, ...,
         -7.3876953e-01,  1.6611328e+00, -7.3828125e-01],
        [-2.6250000e+00, -2.7441406e+00, -1.3740234e+00, ...,
         -1.9980469e+00,  5.4296875e-01,  3.4698486e-0

In [22]:
torch.cuda.empty_cache()
# Evaluate in smaller chunks if dataset is large
eval_results = []
for i in range(0, len(trainer.eval_dataset), trainer.args.eval_batch_size * 4):  # Process in chunks of 4x eval_batch_size
    chunk = trainer.eval_dataset[i : i + trainer.args.eval_batch_size * 4]
#    chunk = trainer.eval_dataset
    chunk_dataset = Dataset.from_dict(chunk)
    chunk_results = trainer.evaluate(eval_dataset=chunk_dataset)
    eval_results.append(chunk_results)

KeyboardInterrupt: 

In [25]:
results = pd.DataFrame.from_dict(eval_results)
results.describe()


,eval_loss,eval_model_preparation_time,eval_rouge1,eval_rouge2,eval_rougeL,eval_rougeLsum,eval_runtime,eval_samples_per_second,eval_steps_per_second
count,900.000000,9.000000e+02,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,0.761236,8.000000e-03,0.783752,0.630682,0.762773,0.771425,2.741370,1.464853,1.464853
std,0.307272,5.207064e-18,0.080676,0.131775,0.089575,0.086797,0.187275,0.085344,0.085344
min,0.050005,8.000000e-03,0.510069,0.256768,0.479062,0.476735,2.475900,0.847000,0.847000
25%,0.539402,8.000000e-03,0.726739,0.534828,0.698756,0.711587,2.632175,1.430000,1.430000
50%,0.751730,8.000000e-03,0.787481,0.634690,0.768197,0.776388,2.708250,1.477000,1.477000
75%,0.961912,8.000000e-03,0.842011,0.728660,0.829966,0.836032,2.797150,1.520000,1.520000
max,1.875449,8.000000e-03,0.978018,0.958971,0.978018,0.978018,4.721800,1.616000,1.616000


In [27]:
results.shape

(900, 9)

In [22]:
print(os.path.join('/content/drive/MyDrive/model', f"{base_model}_{timestamp}"))

/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_20241107_032806


In [9]:
data

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Model,tim-stmp,Epoch,Learning_rate,Lora_rank,Lora_alpha,quant_4bit,quant_8bit,Trainable_parameters,model_path,tokeniser_path,train_loss
0,0,0.0,0.0,0.0,0.0,google/flan-t5-base,20241106_114401,3.0,0.00005,8,8,True,False,NaN,/content/drive/MyDrive/model/google/flan-t5-ba...,/content/drive/MyDrive/model/google/flan-t5-ba...,NaN
1,1,1.0,1.0,1.0,NaN,test123,20241106_114401,3.0,0.00005,8,8,True,False,NaN,/content/drive/MyDrive/model/google/flan-t5-ba...,/content/drive/MyDrive/model/google/flan-t5-ba...,NaN
2,2,2.0,2.0,NaN,NaN,test123,20241107_032806,1.0,0.00010,8,32,True,False,NaN,/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,NaN
3,3,3.0,NaN,NaN,NaN,codellama/CodeLlama-7b-hf,20241107_032806,1.0,0.00010,8,32,True,False,NaN,/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,NaN
4,4,NaN,NaN,NaN,NaN,codellama/CodeLlama-7b-hf,20241107_032806,1.0,0.00010,8,32,True,False,NaN,/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,0.910208


In [52]:
  data = pd.read_csv("/content/drive/MyDrive/dataset/results.csv")
  #data.loc[data['Model']=='codellama/CodeLlama-7b-hf']['tim-stmp'][4]
  data

,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Model,tim-stmp,Epoch,Learning_rate,Lora_rank,Lora_alpha,quant_4bit,quant_8bit,Trainable_parameters,model_path,tokeniser_path,train_loss
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,google/flan-t5-base,20241106_114401,3.0,0.00005,8,8,True,False,NaN,/content/drive/MyDrive/model/google/flan-t5-ba...,/content/drive/MyDrive/model/google/flan-t5-ba...,NaN
1,1,1.0,1.0,1.0,1.0,1.0,1.0,NaN,test123,20241106_114401,3.0,0.00005,8,8,True,False,NaN,/content/drive/MyDrive/model/google/flan-t5-ba...,/content/drive/MyDrive/model/google/flan-t5-ba...,NaN
2,2,2.0,2.0,2.0,2.0,2.0,NaN,NaN,test123,20241107_032806,1.0,0.00010,8,32,True,False,NaN,/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,NaN
3,3,3.0,3.0,3.0,3.0,NaN,NaN,NaN,codellama/CodeLlama-7b-hf,20241107_032806,1.0,0.00010,8,32,True,False,NaN,/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,NaN
4,4,4.0,4.0,4.0,NaN,NaN,NaN,NaN,codellama/CodeLlama-7b-hf,20241107_032806,1.0,0.00010,8,32,True,False,NaN,/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,0.910208
5,5,5.0,5.0,NaN,NaN,NaN,NaN,NaN,codellama/CodeLlama-7b-hf,20241107_150314,1.0,0.00010,16,32,True,False,NaN,/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,0.910208
6,6,6.0,NaN,NaN,NaN,NaN,NaN,NaN,codellama/CodeLlama-7b-hf,20241107_150314,1.0,0.00010,16,32,True,False,NaN,/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,0.908454
7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,codellama/CodeLlama-7b-hf,20241107_150314,1.0,0.00010,16,32,True,False,"trainable params: 39,976,960 || all params: 6,...",/content/drive/MyDrive/model/codellama/CodeLla...,/content/drive/MyDrive/model/codellama/CodeLla...,0.908454


In [37]:
# Get training loss from the last entry in log_history
final_training_loss = trainer.state.log_history[-1]['loss']

KeyError: 'loss'

In [ ]:
config = model.config  # Assuming 'model' is your loaded model
max_seq_length = config.max_position_embeddings
print(f"Maximum sequence length: {max_seq_length}")


/content/drive/MyDrive/model/codellama/CodeLlama-7b-hf_20241106_150056


In [49]:
trainer.train_dataset

Dataset({
    features: ['instruction', 'response', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 18706
})

In [64]:
type(trainer.eval_dataset[1])

dict

In [67]:
torch.cuda.empty_cache()
# Evaluate in smaller chunks if dataset is large
eval_results = []
metric = evaluate.load("rouge") # Load the evaluation metric

for i in range(0, len(trainer.eval_dataset), trainer.args.eval_batch_size * 4):  # Process in chunks of 4x eval_batch_size
    chunk = trainer.eval_dataset[i : i + trainer.args.eval_batch_size * 4] #Get a slice of the dataset as a list of dictionaries
    chunk_dataset = Dataset.from_dict({k: [d[k] for d in chunk] for k in chunk[0].keys()})  #Convert list of dicts to dict of lists

    # Get predictions from the trainer
    predictions = trainer.predict(chunk_dataset)

    # Assuming 'predictions' is a tuple/list like (predictions, labels), extract predictions
    decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

    # Extract references from chunk_dataset
    references = [ex["target"] for ex in chunk_dataset] # Adjust 'target' to the actual key for references

    # Reshape the data to match the expected format
    formatted_results = [{"predictions": pred, "references": ref} for pred, ref in zip(decoded_preds, references)]

    # Calculate the metric for this chunk
    chunk_results = metric.compute(predictions=decoded_preds, references=references)
    eval_results.append(chunk_results)


KeyError: 0

In [72]:
trainer.eval_dataset[1]

dict_keys(['instruction', 'response', '__index_level_0__', 'input_ids', 'attention_mask'])

In [103]:
def compute_metrics(eval_pred):
    """Computes and returns a dictionary of metrics (e.g., ROUGE scores)."""

    # Unpack predictions and labels
    predictions = eval_pred['predictions']
    labels = eval_pred['labels']
    print(predictions)

    # Decode the predictions and labels
    decoded_preds = [tokenizer.decode(logit.argmax(axis=-1)) for logit in logits]
    decoded_labels = [tokenizer.decode(label) for label in labels]

    # Load and compute the ROUGE metric
    metric = evaluate.load("rouge")
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Extract specific ROUGE metrics, if desired
    result = {
        "rouge1": result["rouge1"].mid.fmeasure,
        "rouge2": result["rouge2"].mid.fmeasure,
        "rougeL": result["rougeL"].mid.fmeasure,
    }

    return result


In [104]:
    chunk = trainer.eval_dataset[0:1]
    chunk_dataset = Dataset.from_dict(chunk)
    print(chunk_dataset)
    # Get predictions from the trainer
    trainer.predict(chunk_dataset)


Dataset({
    features: ['instruction', 'response', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 1
})


ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: [[-6.3554688  -7.6289062   3.1992188  ... -0.7402344   1.6601562
  -0.73779297]
 [-2.6308594  -2.7558594  -1.3681641  ... -1.9970703   0.54541016
   0.03485107]
 [-5.3242188  -5.0585938   0.5776367  ...  0.42578125 -0.5317383
   1.1552734 ]
 ...
 [-3.8085938  -1.3515625   7.0429688  ...  0.6723633   0.6459961
  -0.66552734]
 [-2.6035156  -0.40307617  8.4609375  ...  1.1513672   0.65771484
  -1.2060547 ]
 [-2.2851562   0.12054443  7.203125   ...  0.4333496   0.79785156
  -1.109375  ]],
Input references: [    1 14350  5132   775   304  4505   278  3414 29889    13  3492   526
  2183   263  6374  6

In [95]:
chunk['input_ids']

[[1,
  14350,
  5132,
  775,
  304,
  4505,
  278,
  3414,
  29889,
  13,
  3492,
  526,
  2183,
  263,
  6374,
  6043,
  405,
  29889,
  10987,
  278,
  1353,
  310,
  278,
  11000,
  310,
  11920,
  318,
  322,
  325,
  313,
  29900,
  229,
  140,
  169,
  29884,
  29892,
  29894,
  229,
  140,
  169,
  29940,
  29897,
  1316,
  393,
  727,
  1863,
  1023,
  1661,
  29899,
  22198,
  11920,
  263,
  322,
  289,
  24064,
  263,
  921,
  272,
  289,
  29922,
  29884,
  322,
  263,
  29974,
  29890,
  29922,
  29894,
  29889,
  2266,
  29892,
  921,
  272,
  20169,
  278,
  2586,
  3538,
  29192,
  6323,
  29889,
  4001,
  372,
  508,
  367,
  14154,
  2919,
  29892,
  10272,
  278,
  1234,
  878,
  7207,
  29871,
  29896,
  29900,
  29985,
  29929,
  29974,
  29955,
  29889,
  13,
  13,
  27427,
  13,
  13,
  29930,
  29871,
  29896,
  229,
  140,
  169,
  29940,
  229,
  140,
  169,
  29896,
  29900,
  998,
  29896,
  29947,
  29913,
  13,
  13,
  4290,
  13,
  13,
  1576,
  1881,
  3

In [94]:
chunk_dataset = Dataset.from_dict({k: [d[k] for d in chunk if isinstance(d, dict)] for k in chunk[0].keys() if isinstance(chunk[0], dict)})  # Ensures that k and d is a dictionary

KeyError: 0

In [102]:
i = [[-6.3554688  -7.6289062   3.1992188 -0.7402344   1.6601562
  -0.73779297]
 [-2.6308594  -2.7558594  -1.3681641  -1.9970703   0.54541016
   0.03485107]
 [-5.3242188  -5.0585938   0.5776367   0.42578125 -0.5317383
   1.1552734 ]
 [-3.8085938  -1.3515625   7.0429688   0.6723633   0.6459961
  -0.66552734]
 [-2.6035156  -0.40307617  8.4609375   1.1513672   0.65771484
  -1.2060547 ]
 [-2.2851562   0.12054443  7.203125    0.4333496   0.79785156
  -1.109375  ]]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-102-fcd388b842b3>, line 1)

In [110]:
i = torch.randn(4, 4)
j = [logit.argmax(axis=-1) for logit in i]

In [123]:
i

tensor([[ 0.3057, -0.7746,  0.0349,  0.3211],
        [ 1.5736, -0.8455,  1.3123,  0.6872],
        [-1.2347, -0.4879, -1.4181,  0.8963],
        [ 0.0499,  2.2667,  1.1790, -0.4345]])

In [124]:
[i[] for i in [torch.argmax(logit, -1) for logit in i]]

tensor(3)

In [115]:
i[j[0]]

tensor([ 0.0499,  2.2667,  1.1790, -0.4345])

In [108]:
[tokenizer.decode(logit.argmax(axis=-1)) for logit in logits]

NameError: name 'logits' is not defined